Import Packages

In [1]:
from whisper.brain import (
    initialize_from_whisper,
    init_discriminator,
    load_checkpoint,
    WillettDataset,
    FastBigAssTrainer,
    CLIPTrainer,
)
import torch
import pandas as pd
import numpy as np
import torch
import wandb
import os
import random

Initialize model from pretrained weights

# CLIP RUN

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {'CUDA' if DEVICE.type == 'cuda' else 'CPU'}.")
trainset = WillettDataset("K:\ke\sta\data\Willett&EtAl2023\data\Willett&EtAl2023.h5", "train", device=DEVICE)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True)
evalset = WillettDataset("K:\ke\sta\data\Willett&EtAl2023\data\Willett&EtAl2023.h5", "test", device=DEVICE)
evalloader = torch.utils.data.DataLoader(evalset, batch_size=8, shuffle=True)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

model = initialize_from_whisper(name='tiny.en')
model_state_dict = torch.load("K:\ke\sta\data\Willett&EtAl2023\checkpoints\clip-trainer_2024-06-09_16-21-14\\2024-06-09_16-21-14_0_200_whisperbrain_checkpoint.pt")["model"]
model.load_state_dict(model_state_dict)
model.to(DEVICE)

Using CUDA.


WhisperBrain(
  (whisper): Whisper(
    (encoder): AudioEncoder(
      (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
      (blocks): ModuleList(
        (0-3): 4 x ResidualAttentionBlock(
          (attn): MultiHeadAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=False)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (out): Linear(in_features=384, out_features=384, bias=True)
          )
          (attn_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (0): Linear(in_features=384, out_features=1536, bias=True)
            (1): GELU(approximate='none')
            (2): Linear(in_features=1536, out_features=384, bias=True)
          )
          (mlp_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=

In [3]:
def init_scheduler(model, optimizer=None):
    unlocked_params = list(model.brain_encoder.prenet.parameters()) + \
                      list(model.brain_encoder.encoder.conv1.parameters()) + \
                      list(model.brain_encoder.encoder.conv2.parameters()) + \
                      list(model.brain_encoder.encoder.blocks[0].parameters()) + \
                      list(model.brain_encoder.encoder.blocks[-1].parameters())
    if optimizer is None:
        optimizer = torch.optim.Adam(
            unlocked_params,
            lr=4e-5,
            #weight_decay=0.01
        )
    else:
        # just reset the learning rate
        for param_group in optimizer.param_groups:
            param_group['lr'] = 2e-5
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer=optimizer,
        T_max=len(trainloader) - 25,
        eta_min=1e-10
    )
    return optimizer, scheduler

In [4]:

optimizer, scheduler = init_scheduler(model, optimizer=None)
clip_trainer = CLIPTrainer(
        model=model,
        model_optimizer=optimizer,
        checkpoint_dir="K:\ke\sta\data\Willett&EtAl2023\checkpoints",
        checkpoint_interval=25,
        logging_interval=1,
        eval_interval=25,
        forcing_prob=1,
        model_scheduler=scheduler,
        max_grad_norm=1,
        normalizer=None,
        tokenizer=None,
        decoding_options=None,
        device=DEVICE,
        eval_sample_size=15
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: olivershetler. Use `wandb login --relogin` to force relogin


In [5]:
for i in range(20):
    optimizer, scheduler = init_scheduler(model, optimizer=optimizer)
    clip_trainer.model_optimizer, clip_trainer.model_scheduler = optimizer, scheduler
    fps = torch.arange(1, 0.5, -0.5 / len(trainloader))
    for k, batch in enumerate(trainloader):
        clip_trainer.forcing_prob = fps[k].item()
        clip_trainer._step_epoch(k, batch, evalloader, epoch=i)
        if k > 25:
            clip_trainer.model_scheduler.step()

k:\ke\dev\WhisperBrain\.venv\Lib\site-packages\torch\nn\modules\conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv3d(


k:\ke\dev\WhisperBrain\.venv\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


PermissionError: [Errno 13] Permission denied: 'K:\\ke\\sta\\data\\Willett&EtAl2023\\checkpoints\\clip-trainer_2024-06-11_15-14-40\\2024-06-11_15-14-40_eval_results.csv'

In [ ]:
encoder_model_optimizer = torch.optim.Adam(
    model.brain_encoder.parameters(),
    lr=10e-4/40)
encoder_model_scheduler = torch.optim.lr_scheduler.LinearLR(
    optimizer=encoder_model_optimizer,
    start_factor=1,
    end_factor=0,
    total_iters=20,
)
clip_encoder_trainer = CLIPTrainer(
        model=model,
        model_optimizer=encoder_model_optimizer,
        checkpoint_dir="K:\ke\sta\data\Willett&EtAl2023\checkpoints",
        checkpoint_interval=25,
        logging_interval=1,
        eval_interval=25,
        forcing_prob=1,
        model_scheduler=None,
        max_grad_norm=1,
        normalizer=None,
        tokenizer=None,
        decoding_options=None,
        device=DEVICE,
        eval_sample_size=15
)

In [ ]:
fpu = 0.8
fpl = 0.7
for i in range(1, 30):
    # lower the forcing prob lower bound until it reaches 0
    if fpl > 0:
        fpl = 0.7-i*0.1
    # lower the forcing prob upper bound every 4 epochs
    if fpu > 0:
        fpu = 0.8-i*0.05
    clip_encoder_trainer.train(trainloader, evalloader=evalloader, epoch=i, forcing_probs=[fpu, fpl])
    encoder_model_scheduler.step()
    print(encoder_model_optimizer.param_groups[0]['lr'])

# FINAL RUN

In [ ]:
model = initialize_from_whisper(name='tiny.en')
embedding_critic = init_discriminator(model.dims, 'embedding')
logit_critic = init_discriminator(model.dims, 'logits')

model_state_dict = torch.load("K:\ke\sta\data\Willett&EtAl2023\checkpoints\\big-ass-trainer_2024-05-31_18-01-01\\2024-05-31_18-01-01_0_600_whisperbrain_checkpoin_0_600.pt")["model"]
model.load_state_dict(model_state_dict)

embedding_critic_state_dict = torch.load("K:\ke\sta\data\Willett&EtAl2023\checkpoints\\big-ass-trainer_2024-05-31_18-01-01\\2024-05-31_18-01-01_0_600_logit_critic_checkpoint.pt")["critic"]

logit_critic_state_dict = torch.load("K:\ke\sta\data\Willett&EtAl2023\checkpoints\\big-ass-trainer_2024-05-31_18-01-01\\2024-05-31_18-01-01_0_600_embedding_critic_checkpoint.pt")["critic"]

print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)
print(f"Embedding critic has {sum(np.prod(p.shape) for p in embedding_critic.parameters()):,} parameters.")
print(f"Logit critic has {sum(np.prod(p.shape) for p in logit_critic.parameters()):,} parameters.")

model.to(DEVICE)
for param in model.whisper.parameters():
    param.requires_grad = False
torch.autograd.set_detect_anomaly(True)

embedding_critic.to(DEVICE)
logit_critic.to(DEVICE)
forcing_prob = 1
learning_rate = 0.0001

# set up parallelization


prenet_model_optimizer = torch.optim.Adam(
    model.brain_encoder.prenet.parameters(),
    lr=learning_rate)
brain_encoder_model_optimizer = torch.optim.Adam(
    model.brain_encoder.parameters(),
    lr=learning_rate)
embedding_critic_optimizer = torch.optim.Adam(
    embedding_critic.parameters(),
    lr=learning_rate)
logit_critic_optimizer = torch.optim.Adam(
    logit_critic.parameters(),
    lr=learning_rate)


prenet_trainer = FastBigAssTrainer(
                model=model,
                model_optimizer=prenet_model_optimizer,
                model_loss_weights={
                    "embedding_g_loss": 1,
                    "sup_embedding_loss": 1,
                    "logit_g_loss": 1,
                    "sup_logit_loss": 1,
                    "seq_ce_loss": 0.1
                    },
                embedding_critic=embedding_critic,
                embedding_critic_optimizer=embedding_critic_optimizer,
                logit_critic=logit_critic,
                logit_critic_optimizer=logit_critic_optimizer,
                checkpoint_dir="K:\ke\sta\data\Willett&EtAl2023\checkpoints",
                default_forcing_prob=1,
                checkpoint_interval=200,
                logging_interval=10,
                eval_interval=200,
                model_scheduler=None,
                embedding_critic_scheduler=None,
                logit_critic_scheduler=None,
                critic_scheduler=None,
                max_grad_norm=1,
                normalizer=None,
                tokenizer=None,
                decoding_options=None,
                device=None,
                eval_sample_size=15
)

Model is English-only and has 45,115,760 parameters.
Embedding critic has 622,952 parameters.
Logit critic has 13,705,729 parameters.


In [ ]:
forcing_prob = 1
for epoch in range(10):
    prenet_trainer.train(trainloader, evalloader, epoch, forcing_prob=forcing_prob)
    forcing_prob -= 0.5

k:\ke\dev\WhisperBrain\.venv\Lib\site-packages\torch\nn\modules\conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv3d(
k:\ke\dev\WhisperBrain\.venv\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


KeyboardInterrupt: 